# A WORKING NOTEBOOK TO PRACTICE MY ACQUIRE FUNCTION

In [1]:
# imports needed to make my functions work properly

In [18]:
import os
import pandas as pd 
import numpy as np

# import re

# from sklearn.model_selection import train_test_split
# import sklearn.preprocessing

# sql ace credentials
import env
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer

In [11]:
# setting connectiong to sequel server using env

def get_connection(db, user=env.username, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

#----------------------------------------------
 
# acquiring zillow data using a get_connection

def get_zillow_data(get_connection):
    filename = "zillow.csv"
    
    # if the file already exist , return it. if not, then continue down the line.
    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col = 0)
    
    else:
        
    # read the SQL query into a dataframe
        df = pd.read_sql(
        '''
          select

bedroomcnt,
bathroomcnt, 
calculatedfinishedsquarefeet,
taxvaluedollarcnt,
yearbuilt,
taxamount,
fips

from properties_2017 
          
join propertylandusetype on propertylandusetype.propertylandusetypeid = 

properties_2017.propertylandusetypeid

where propertylandusedesc = "Single Family Residential"


        ''', get_connection('zillow'))
        
        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)
        
        
        return df


In [4]:
# call function to gather data
df = get_zillow_data(get_connection)

In [5]:
# a picture at what we are working with
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [6]:
# see how many observations and how many features
df.shape

(2152863, 7)

In [7]:
# let's look at what changes might be appropriate
df.dtypes

bedroomcnt                      float64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                       float64
taxamount                       float64
fips                            float64
dtype: object

In [8]:
# bathroom count has half values and tax amount has decimals amounts
df.dtypes


bedroomcnt                      float64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                       float64
taxamount                       float64
fips                            float64
dtype: object

In [9]:
df = df.dropna()
    
    # change datatypes
df['bedroomcnt'] = df['bedroomcnt'].astype(int)
df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
df["taxvaluedollarcnt"] = df["taxvaluedollarcnt"].astype(int)
df['yearbuilt'] = df['yearbuilt'].astype(int)
df['fips'] = df['fips'].astype(int)
    

In [10]:
df = df.rename(columns={'bedroomcnt':'bedroom', 
                            'bathroomcnt':'bathroom', 
                            'calculatedfinishedsquarefeet':'sq_feet', 
                            'taxvaluedollarcnt':'tax_value',
                            'yearbuilt':'year_built',
                            'taxamount':'tax_amount'})


In [12]:
def change_zillow(df):
    
    ''' takes in a zillow dataframe and returns a cleaned version''' 
    
    # for now im going to drop columns with nulls. The percent dropped still leaves me with 98% + of my data
    df = df.dropna()
    
    # change datatypes
    df['bedroomcnt'] = df['bedroomcnt'].astype(int)
    df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
    df["taxvaluedollarcnt"] = df["taxvaluedollarcnt"].astype(int)
    df['yearbuilt'] = df['yearbuilt'].astype(int)
    df['fips'] = df['fips'].astype(int)
    
    return df 

In [13]:
def clean_zillow():
    # call function to gather data
    df = get_zillow_data(get_connection)
    
    # changes col names, drops nulls, changes data types
    df = change_zillow(df)
    
    return df

In [8]:
df = clean_zillow()

/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_6649/83728904.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bedroomcnt'] = df['bedroomcnt'].astype(int)
/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_6649/83728904.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_6649/83728904.py:11: SettingWithCopyWarning: 
A value is trying to be 

In [19]:
def split_data(df):
    '''
    take in a DataFrame and return train, validate, and test DataFrames.'''
    
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123)
    return train, validate, test



In [20]:
def wrangle_zillow():
    # calls a function to return workable dataframe
    df = clean_zillow()
    
    train,validate,test = split_data(df)
    
    return df, train, validate, test
    

In [21]:
df,train,validate,test = wrangle_zillow()

/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_6649/83728904.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bedroomcnt'] = df['bedroomcnt'].astype(int)
/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_6649/83728904.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_6649/83728904.py:11: SettingWithCopyWarning: 
A value is trying to be 

In [24]:
train.shape, validate.shape, test.shape

((1198531, 7), (513657, 7), (428047, 7))

In [25]:
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4,2.0,3633,296425,2005,6941.39,6037
6,3,4.0,1620,847770,2011,10244.94,6037
7,3,2.0,2077,646760,1926,7924.68,6037
11,0,0.0,1200,5328,1972,91.60,6037
14,0,0.0,171,6920,1973,255.17,6037
...,...,...,...,...,...,...,...
2152856,4,4.0,4375,422400,2015,13877.56,6037
2152858,4,3.0,2262,960756,2015,13494.52,6059
2152859,4,4.5,3127,536061,2014,6244.16,6059
2152861,3,2.5,1974,424353,2015,5302.70,6059


In [ ]:
plt.figure(figsize=16,6)

cols = [bedroom]